# Python Sequitur (Nevill-Manning) algorithm

For the algorithm reference see:

1. http://arxiv.org/abs/cs/9709102
2. https://en.wikipedia.org/wiki/Sequitur_algorithm
3. http://www.sequitur.info/

I'm testing the pysequitur library against the web interface provided by sequitur.info.

## Background

Main reason to make a new library is that I needed to have the Sequitur algorithm to work with "any" object pairs (digram), not just textual input. This is best achieved by working with list indexes rather than rudimentary text replacements on the algorithm.

Other reason was to test efficiency of two different approaches for the algorithm:

1. array slicing method
2. digram storage method

I was quessing array slicer is an easier way, but searching digrams from existing rules might take longer time and concequently have a downside effect to the performance of the algorithm. Thus the second way, "digram storage method" was implemented. Trivial tests show that the latter is a faster implementation indeed.

Other Python libraries for the algorithm can be found from GitHub:

1. https://github.com/mspandit/sequitur-python
2. https://github.com/raysaagar/parallel-sequitur

The latter provides both serial and parallel versions of the algorithm. Unexpectively the parallel version is slower on small data sets on my tests. Both algos work slower than the "digram storage method". I found _sequitur-python_ just recently so performance and features of that library are yet to be tested.


## Array slicing method

I'm not going to the deep details of the source code on these two implementations. Basicly on array slicing method every rule is a plain list of sequence items. So finding and comparing digrams needs to loop over all rules and then slice root sequence to parts, which are two items long. Comparison is thus made like this:

<pre><code class="python">rule[i:i+2] == digram</code></pre>

This is one of the most resource eating part on the algorithm. Of cource the rule utility is another similar loop over loop over loop place which takes most of the time on the algo:

<pre><code class="python">for rule1 in self:
    for rule2 in self:
        l = len(rule2)
        ind = 0
        while ind < l:
            c = rule2[ind+1:]
            del rule2[ind:]
            rule2 += rule1 + c
            ind += 1
</code></pre>

## Digram storage method

On the digram storage method array slicing is eliminated by storing every item with the previous item. In this case digrams are stored on memory, which causes approximately twice the memory consumption but then, at least trice the performance increase. On Digram storage method comparison can be done with more efficient way:

<pre><code class="python">digram in rule</code></pre>

Also the rule utility is behaving a bit different way after this data structure change. Still I coudn't find a way to eliminate loops over loops, but rule modification method seems to be more efficient, because there is no need to delete and reinstantiate the list of rule items. Simplified it looks like this:

<pre><code class="python">for rule1 in self:
    for rule2 in self:
        l = len(rule2)
        ind = 0
        while ind < l:
            self[ind:ind] = rule[:]
</code></pre>

All code with comments can be found from GitHub directory: https://github.com/markomanninen/pysequitur/blob/master/pysequitur/main.py

## Using library

Library offers these two implementations of the algorithm with dedicated classes:

- Sequencer
- Sequencer2

For nice output print_grammar function is provided. Five different constants can be used to alter output and representation of the classes used on the library's print utility:

- **RULE_INDEX_STR**<br/>
  How to represent rule indexes on output, default is "^%s" where %s is replaced with a rule index number
- **SEQUENCE_KEY**<br/>
  Define special key to be used to mark the main root sequence, default is "S"
- **ARROW**<br/>
  Special character to be used as a separator between rule indices and rules, default "→"
- **NEWLINE_REPLACEMENT**<br/>
  What to do with newlines on output, default replacement is "↵"
- **SPACE_REPLACEMENT**<br/>
  What to do with space characters on output, default replacement is "_"

## Comparison

Let's compare these two different sequencers and print_grammar utility with an example children nursery rhyme:

In [1]:
from pysequitur import Sequencer, Sequencer2, print_grammar

seq = """pease porridge hot,
pease porridge cold,
pease porridge in the pot,
nine days old.

some like it hot,
some like it cold,
some like it in the pot,
nine days old."""

In [2]:
print (Sequencer(seq).get())

[[[None, ^3], [^3, ^9], [^9, ^5], [^5, ^11], [^11, ^5], [^5, ^7], [^7, '\n'], ['\n', '\n'], ['\n', ^10], [^10, ^9], [^9, ^12], [^12, ^11], [^11, ^12], [^12, ^7]], [['e', ' ']], [[',', '\n']], [['p', 'e'], ['e', 'a'], ['a', 's'], ['s', ^4], [^4, 'r'], ['r', 'r'], ['r', 'i'], ['i', 'd'], ['d', 'g'], ['g', ^1]], [[^1, 'p'], ['p', 'o']], [[^2, ^3]], [['i', 'n']], [[^6, ' '], [' ', 't'], ['t', 'h'], ['h', ^4], [^4, 't'], ['t', ^2], [^2, 'n'], ['n', ^6], [^6, ^1], [^1, 'd'], ['d', 'a'], ['a', 'y'], ['y', 's'], ['s', ' '], [' ', ^8], [^8, '.']], [['o', 'l'], ['l', 'd']], [['h', 'o'], ['o', 't']], [['s', 'o'], ['o', 'm'], ['m', ^1], [^1, 'l'], ['l', 'i'], ['i', 'k'], ['k', ^1], [^1, 'i'], ['i', 't'], ['t', ' ']], [['c', ^8]], [[^2, ^10]], None, None]


In [3]:
print (Sequencer2(seq).get())

[[^3, ^7, ^4, ^5, ^4, ^11, '\n', '\n', ^8, ^7, ^9, ^5, ^9, ^11], ['e', ' '], ['i', 'n'], ['p', 'e', 'a', 's', ^16, 'r', 'r', 'i', 'd', 'g', ^1], [^17, ^3], ['c', ^6], ['o', 'l', 'd'], ['h', 'o', 't'], ['s', 'o', 'm', ^1, 'l', 'i', 'k', ^1, 'i', 't', ' '], [^17, ^8], None, [^2, ' ', 't', 'h', ^16, 't', ^17, 'n', ^2, ^1, 'd', 'a', 'y', 's', ' ', ^6, '.'], None, None, None, None, [^1, 'p', 'o'], [',', '\n'], None, None, None, None, None, None, None, None, None, None, None, None]


As we can notice, data structures are different, latter is somewhat easier to understand, more straightforward. Now let's compare grammar generated by sequencers:

In [4]:
print_grammar(Sequencer(seq))

S → ^3^9^5^11^5^7↵↵^10^9^12^11^12^7
1 → e_
2 → ,↵
3 → peas^4rridg^1
4 → ^1po
5 → ^2^3
6 → in
7 → ^6_th^4t^2n^6^1days ^8.
8 → old
9 → hot
10 → som^1lik^1it 
11 → c^8
12 → ^2^10


In [5]:
print_grammar(Sequencer2(seq))

S → ^3^7^4^5^4^11↵↵^8^7^9^5^9^11
1 → e_
2 → in
3 → peas^16rridg^1
4 → ^17^3
5 → c^6
6 → old
7 → hot
8 → som^1lik^1it_
9 → ^17^8
11 → ^2_th^16t^17n^2^1days_^6.
16 → ^1po
17 → ,↵


They should give similar output, just index numbers should be different. Then let's make some trivial efficiency check with __%%timeit__ notebook magics:

In [6]:
%%timeit
Sequencer(seq)

100 loops, best of 3: 3.82 ms per loop


In [7]:
%%timeit
Sequencer2(seq)

100 loops, best of 3: 9.82 ms per loop


Sequencer 2 takes more than twice longer to create the grammar. Before trying with longer text samples, let's try a trivial tester, which compares generated sequence to the expected output. If they don't match, function will output error.

### Test output

In [8]:
def test(sequence, expect, sequencer_class):
    """ Test sequence if result is as expected. This is a very trivial tester. """
    s = sequencer_class()
    for c in sequence:
        s.stream(c)
    try:
        assert str(s) == expect
        print ("Test ok!")
    except AssertionError:
        print ('Assertion error!', 'Sequence "%s" gave: "%s". Expected: "%s"' % (sequence, str(s), expect))

In [9]:
test('abcdbcabcd', '^3^1^3', Sequencer)

Test ok!


In [10]:
test('abcdbcabcdbc', '^3^1^3', Sequencer)

Assertion error! Sequence "abcdbcabcdbc" gave: "^2^2". Expected: "^3^1^3"


### Longer text sample

Now it is time to compare longer text with sequencers. I'm using samples found from: http://www.sequitur.info/ to double check everything works as expeted. First import Genesis text from file:

In [11]:
input_string = open('genesis1.txt', 'r').read()

Then processing with two different sequencers:

In [12]:
%%timeit
Sequencer(input_string)

1 loops, best of 3: 364 ms per loop


In [13]:
%%timeit
Sequencer2(input_string)

1 loops, best of 3: 1.13 s per loop


Result is that "digram storage method" seems to be trice faster than simpler "array slicing method".

Other long text sample ("I am Sam" poem) looks like this when converted to a grammar:

In [14]:
input_string = open('IamSam.txt', 'r').read()
s = Sequencer(input_string)
print_grammar(s)

S → ^3↵^3^2_^5↵^6^6!^14t^8^12D^9^19^33^12^32^36.^14^52^37^22^84^28?^12^29^27^28^31^29^53^33^34^36^37^43^41^39↵^42^41^55^43^49^22^42^49↵^112↵^53^79^12^61^59b^64^61↵^67^64↵^104b^71^67^71^66^106^171^74^75^74^76^77^141^79^156?_^123^60^90^60^83_^83↵^149^58^230^105^99^88_^178^87^90^12^96^214^96^59^98^60^18^69^103^135^103^104^132^193^105^165^107^136^51^194^55^107^106^113m^108^110^142^117_^112^176^34^52^117^150^120^120^177↵^128^60^130^128^190^2^119L^134^189^135^115^169^139^186^137i^138f^65^147^142^153^213^145^192^106^174^143^76^175^150^12^216^35^48^151^60^149_^154^152^182^115^154^161^87^154^31^156^157^87^160^161^115^160^191^170^128^87^163^102^115^166^95^165^217^115^200^168^163^106^170^169^171^170^172^174^175^176^181^201^244^60^113^69^22^110^199^177^87^180^179^178^31^180^181^182^87^187^60↵^7^186^115^187^188^185^140^185^180^143^160^143^128^191^190^193^189^165^192^136^175^194^143^233^175^142^195^75^195^203^239^196↵^197ss↵^30^198^196^46^199↵^201^82^236O^210^205^206^119^202^152^203^207^206^114^207^

### Stream sequence

So far we have provided a input sequence as a single pass as a class constructor argument. Let's assume, that sequencer is used to find out repeating hierarchal structures from a constant stream of a numeric input data. On the next code block I will emulate constant data flow within a while loop. A small delay is used on a loop to create an effect of a continuous stream of timely data.

To differentiate rule indexes and numeric data I will change RULE_INDEX_STR constant to the more approriate one so that rule index numbers will be wrapped with curly brackets. Also because index numbers in Python are in a form of integer types, we need to make sure that Rule Indices and input values are not mixed on the process. Class Int is presented here for that reason.

_Use Kernel->Interrupt from the main menu to stop while loop in case of infinite recursion!_

In [15]:
import time
from IPython.display import clear_output
import pysequitur

class Int(int):
    def __repr__(self):
        return str(self)
    def __eq__(self, v):
        return isinstance(v, Int) and int.__eq__(self, v)

# reinstantiate default rule index string: ^%s with: {%s}
pysequitur.main.RULE_INDEX_STR = "{%s}"
# init sequencer
s = Sequencer()
# init delay, 0.5 seconds
delay = 0.5
# init numeric data and map them to Int type
a = [1,2,3,4,2,3,1,2,3,4,1,2,3,4,2,3,1,2,3,4]
a = list(map(Int, a))

try:
    l = len(a)
    i = 0
    while i < l:
        s.stream(a[i])
        print_grammar(s)
        time.sleep(delay)
        clear_output(wait=True)
        i+=1
except KeyboardInterrupt:
    pass

S → {5}{5}
1 → 23
3 → 1{1}4
5 → {3}{1}{3}


## Resolve

When sequencer is used to generate (which also means same as compress the data), we can use resolve method of the Sequencer class to decode main rule back to original sequence:

In [16]:
input_string = open('peaseporridge.txt', 'r').read()
s = Sequencer(input_string)
print (''.join(s.resolve()))

pease porridge hot,
pease porridge cold,
pease porridge in the pot,
nine days old.

some like it hot,
some like it cold,
some like it in the pot,
nine days old.


## Random sequence

One more final case just to demonstrate sequencer in working is to use random letter pattern and see how it is processed by the library:

In [17]:
import time
from random import shuffle, randint as rand
from IPython.display import clear_output

def test_random_seq(f=5, delay=0.1):
    abc = 'abcdefghijklmnopqrstuvwyz'
    l = len(abc)-1
    s = ''
    i = l*f
    # to stop script execution on Jupyter notebook: Kernel -> Interrupt
    try:
        s = Sequencer(abc[rand(0, l)])
        while i > 0:
            i -= 1
            s.stream(abc[rand(0, l)])
            print ('sequence:', s)
            print ('grammar:', s.get()[0])
            print ('rules:', s.get()[1:])
            print ('resolve:', s.resolve())
            time.sleep(delay)
            clear_output(wait=True)
    except KeyboardInterrupt:
        pass

In [18]:
test_random_seq()

sequence: {1}{2}u{1}ffypkfcjmprw{4}dbeyb{2}qzhuijztwcowugbomedvcsa{3}mzzcwnolulpzpsz{1}jokdodmjhvl{3}wjnpctdnyucngtpniswp{4}vhckoqavykmvaica
grammar: [[None, {1}], [{1}, {2}], [{2}, 'u'], ['u', {1}], [{1}, 'f'], ['f', 'f'], ['f', 'y'], ['y', 'p'], ['p', 'k'], ['k', 'f'], ['f', 'c'], ['c', 'j'], ['j', 'm'], ['m', 'p'], ['p', 'r'], ['r', 'w'], ['w', {4}], [{4}, 'd'], ['d', 'b'], ['b', 'e'], ['e', 'y'], ['y', 'b'], ['b', {2}], [{2}, 'q'], ['q', 'z'], ['z', 'h'], ['h', 'u'], ['u', 'i'], ['i', 'j'], ['j', 'z'], ['z', 't'], ['t', 'w'], ['w', 'c'], ['c', 'o'], ['o', 'w'], ['w', 'u'], ['u', 'g'], ['g', 'b'], ['b', 'o'], ['o', 'm'], ['m', 'e'], ['e', 'd'], ['d', 'v'], ['v', 'c'], ['c', 's'], ['s', 'a'], ['a', {3}], [{3}, 'm'], ['m', 'z'], ['z', 'z'], ['z', 'c'], ['c', 'w'], ['w', 'n'], ['n', 'o'], ['o', 'l'], ['l', 'u'], ['u', 'l'], ['l', 'p'], ['p', 'z'], ['z', 'p'], ['p', 's'], ['s', 'z'], ['z', {1}], [{1}, 'j'], ['j', 'o'], ['o', 'k'], ['k', 'd'], ['d', 'o'], ['o', 'd'], ['d', 'm'], ['m', 'j

## The [MIT](https://github.com/markomanninen/pysequitur/blob/master/LICENSE) License

Copyright &copy; 2016 Marko Manninen